In [13]:
import spotipy
import pandas as pd

from sqlalchemy import create_engine
from spotipy.oauth2 import SpotifyOAuth

from config import CLIENT_ID, CLIENT_SECRET, REDIRECT_URI, SCOPE, DATABASE_URL

from utils import generate_short_uuid
from utils import log_exception

In [14]:
# ETL functions related to user's top artists

def extract_top_artists():

    try:
        # Authentication with Spotify API
        sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                                       client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=SCOPE))
        raw_data = sp.current_user_top_artists()
        return raw_data
    except Exception as e:
        log_exception("Error while extracting user's top artists data:", e)
        return None


def transform_top_artists(raw_data):

    try:
        user_top_artists = raw_data["items"]

        artists_dict = {
            "id": [],
            "name": [],
            "popularity": []
        }

        genres_dict = {
            "id": [],
            "genre": []
        }

        artists_genres_dict = {
            "artist_id": [],
            "genre_id": []
        }

        for artist in user_top_artists:
            artist_id = generate_short_uuid()
            artist_name = artist["name"]
            artist_popularity = artist["popularity"]

            artists_dict["id"].append(artist_id)
            artists_dict["name"].append(artist_name)
            artists_dict["popularity"].append(artist_popularity)

            for genre in artist["genres"]:

                genre_id = ""
                if genre not in genres_dict["genre"]:
                    genre_id = generate_short_uuid()

                    genres_dict["id"].append(genre_id)
                    genres_dict["genre"].append(genre)

                else:
                    index = genres_dict["genre"].index(genre)
                    genre_id = genres_dict["id"][index]

                artists_genres_dict["artist_id"].append(artist_id)
                artists_genres_dict["genre_id"].append(genre_id)

        artists_df = pd.DataFrame(data=artists_dict)
        genres_df = pd.DataFrame(data=genres_dict)
        artists_genres_df = pd.DataFrame(data=artists_genres_dict)

        data = [
            {
                "name": "user_top_artists",
                "df": artists_df,
            },
            {"name": "user_music_genres", "df": genres_df},
            {"name": "user_top_artists_genres", "df": artists_genres_df}
        ]

        return data
    except Exception as e:
        log_exception("Error while transforming user's top artists data:", e)
        return None


def load_top_artists(transformed_data):

    success = False
    try:
        engine = create_engine(url=DATABASE_URL)

        for table_dict in transformed_data:
            table_name = table_dict["name"]

            table_dict["df"].to_sql(
                table_name, engine, if_exists="replace", index=False)

        success = True
    except Exception as e:
        log_exception("Error while loading user's top artists data:", e)
        return None
    finally:
        return success


def run_top_artists_etl():

    success = False
    try:
        raw_data = extract_top_artists()

        transformed_data = transform_top_artists(raw_data)

        success = load_top_artists(transformed_data)

    except Exception as e:
        log_exception(
            "Error while attempting to run user's top artists ETL:", e)
    finally:
        return success

In [18]:
# ETL functions related to user's top songs

def extract_top_songs():
    try:
        sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                                   client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=SCOPE))

        raw_data = sp.current_user_top_tracks()

        return raw_data
    except Exception as e:
        log_exception("Error while attempting to extract user's top songs: ", e)
        return None



def transform_top_songs(raw_data):
    try:
        user_top_songs = raw_data["items"]

        songs_dict = {
            "id": [],
            "name": [],
            "popularity": [],
            # "release_date": []
        }
        for song in user_top_songs:
            song_id = generate_short_uuid()
            song_name = song["name"]
            song_popularity = song["popularity"]
            # song_release_date = song["release_date"]

            songs_dict["id"].append(song_id)
            songs_dict["name"].append(song_name)
            songs_dict["popularity"].append(song_popularity)
            # songs_dict["release_date"].append(song_release_date)

        songs_df = pd.DataFrame(data=songs_dict)

        return songs_df
    except Exception as e:
        log_exception(
            "Error while attempting to transform user's top songs: ", e)
        return None


def load_top_songs(transformed_data):
    success = False
    try:
        engine = create_engine(url=DATABASE_URL)

        transformed_data.to_sql(name="user_top_songs",
                            con=engine, if_exists="replace", index=False)
        success = True
    except Exception as e:
        log_exception("Error while attempting to load user's top songs: ", e)
    finally:
        return success

def run_top_songs_etl():
    success = False
    try:
        raw_data = extract_top_songs()

        transformed_data = transform_top_songs(raw_data)

        success = load_top_songs(transformed_data)
    except Exception as e:
        log_exception("Error while attempting to run user's top songs ETL: ", e)
    finally:
        return success

In [19]:
run_top_artists_etl()
run_top_songs_etl()

True